In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [6]
atmpro = mls
tsfc = 294


...

Best-fit values for each (gas, band)
------------------------------------
o h2o band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.66, 1.66, 1.85)]
  wgt = [(0.5, 0.55, 0.85)]
o co2 band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.000755                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000766
0.000750      2.0 -0.000757                  2.0 -0.000790
0.001052      3.0 -0.000761                  3.0 -0.000767
0.001476      4.0 -0.000765                  4.0 -0.000771
0.002070      5.0 -0.000768                  5.0 -0.000776
0.002904      6.0 -0.000771                  6.0 -0.000780
0.004074      7.0 -0.000773                  7.0 -0.000784
0.005714      8.0 -0.000774                  8.0 -0.000788
0.008015      9.0 -0.000775                  9.0 -0.000790
0.011243     10.0 -0.000775                 10.0 -0.000795
0.015771     11.0 -0.000775                 11.0 -0.000802
0.022122     12.0 -0.000774                 12.0 -0.000808
0.031031     13.0 -0.000772                 13.0 -0.000811
0.043528     14.0 -0.000769                 14.0 -0.000815
0.061057     15.0 -0.000754                 15.0 -0.000821
0.085645     16.0 -0.000662                 16.0 -0.000797
0.120136     17.0 -0.000348                 17.0 -0.000645
0.168516     18.0  0.000314                 18.0 -0.000266
0.236378     19.0  0.001511                 19.0  0.000461
0.331549     20.0  0.003506                 20.0  0.001698
0.465100     21.0  0.006652                 21.0  0.003697
0.652400     22.0  0.011390                 22.0  0.006998
0.915100     23.0  0.016555                 23.0  0.010690
1.283650     24.0  0.016679                 24.0  0.010909
1.800600     25.0  0.011979                 25.0  0.007893
2.525700     26.0  0.007478                 26.0  0.004932
3.542800     27.0  0.004200                 27.0  0.002709
4.969550     28.0  0.002005                 28.0  0.001184
6.970850     29.0  0.000636                 29.0  0.000200
9.778100     30.0 -0.000197                 30.0 -0.000425
13.715850    31.0 -0.000709                 31.0 -0.000825
19.239350    32.0 -0.000999                 32.0 -0.001059
26.987250    33.0 -0.001132                 33.0 -0.001172
37.855300    34.0 -0.001193                 34.0 -0.001231
53.100050    35.0 -0.001230                 35.0 -0.001266
73.887500    36.0 -0.001254                 36.0 -0.001286
97.662500    37.0 -0.001283                 37.0 -0.001311
121.437500   38.0 -0.001322                 38.0 -0.001329
145.212500   39.0 -0.001359                 39.0 -0.001313
168.987500   40.0 -0.001407                 40.0 -0.001335
192.762500   41.0 -0.001500                 41.0 -0.001392
216.537500   42.0 -0.001611                 42.0 -0.001544
240.312500   43.0 -0.001667                 43.0 -0.001659
264.087500   44.0 -0.001620                 44.0 -0.001707
287.862500   45.0 -0.001396                 45.0 -0.001651
311.637500   46.0 -0.000936                 46.0 -0.001395
335.412500   47.0 -0.000216                 47.0 -0.000900
359.187500   48.0  0.000831                 48.0 -0.000146
382.962500   49.0  0.002270                 49.0  0.000952
406.737500   50.0  0.004162                 50.0  0.002472
430.512500   51.0  0.006554                 51.0  0.004465
454.287500   52.0  0.009323                 52.0  0.006907
478.062500   53.0  0.012514                 53.0  0.009861
501.837500   54.0  0.016222                 54.0  0.013423
525.612500   55.0  0.020489                 55.0  0.017481
549.387500   56.0  0.025355                 56.0  0.022220
573.162500   57.0  0.030879                 57.0  0.027696
596.937500   58.0  0.038498                 58.0  0.035232
620.712500   59.0  0.047196                 59.0  0.043960
644.487500   60.0  0.056634                 60.0  0.053324
668.262500   61.0  0.066782                 61.0  0.063221
692.037500   62.0  0.077563                 62.0  0.073679
715.812500   63.0  0.088988                 63.0  0.084750
739.587500   64.0  0.101070                 64.0  0.096584
76

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -60.098092  0.000000e+00 -60.098092        NaN   
1.000000e-08 1            NaN           NaN        NaN -60.167730   
6.244000e-04 2     -60.098092  3.228642e-08 -60.098092 -60.167730   
8.759000e-04 3     -60.098092  4.552040e-08 -60.098092 -60.167730   
1.228600e-03 4     -60.098092  6.477995e-08 -60.098092 -60.167730   
1.723400e-03 5     -60.098092  9.287635e-08 -60.098092 -60.167730   
2.417400e-03 6     -60.098092  1.338518e-07 -60.098092 -60.167730   
3.390900e-03 7     -60.098092  1.936090e-07 -60.098092 -60.167730   
4.756500e-03 8     -60.098093  2.807487e-07 -60.098092 -60.167731   
6.672000e-03 9     -60.098093  4.077944e-07 -60.098092 -60.167731   
9.358900e-03 10    -60.098093  5.930048e-07 -60.098093 -60.167731   
1.312780e-02 11    -60.098094  8.629729e-07 -60.098093 -60.167732   
1.841450e-02 12    -60.098095  1.256442e-06 -60.098094 -60.167733   
2.583020e-02 13    -60.098096  1.829843e-06 -60.098094 -60.167734   
3.623230e-02 14    -60.098098  2.665381e-06 -60.098095 -60.167735   
5.082340e-02 15    -60.098100  3.885774e-06 -60.098096 -60.167738   
7.129060e-02 16    -60.098104  5.812379e-06 -60.098098 -60.167741   
1.000000e-01 17    -60.098110  9.499050e-06 -60.098101 -60.167747   
1.402710e-01 18    -60.098120  1.741495e-05 -60.098102 -60.167756   
1.967600e-01 19    -60.098134  3.430469e-05 -60.098100 -60.167770   
2.759970e-01 20    -60.098156  6.986083e-05 -60.098086 -60.167791   
3.871000e-01 21    -60.098183  1.435771e-04 -60.098040 -60.167821   
5.431000e-01 22    -60.098211  2.941798e-04 -60.097917 -60.167858   
7.617000e-01 23    -60.098218  5.956169e-04 -60.097622 -60.167897   
1.068500e+00 24    -60.098166  1.145355e-03 -60.097020 -60.167920   
1.498800e+00 25    -60.098078  1.908093e-03 -60.096170 -60.167948   
2.102400e+00 26    -60.098044  2.730349e-03 -60.095314 -60.168042   
2.949000e+00 27    -60.098113  3.549114e-03 -60.094564 -60.168239   
4.136600e+00 28    -60.098319  4.346064e-03 -60.093973 -60.168568   
5.802500e+00 29    -60.098697  5.119628e-03 -60.093577 -60.169059   
8.139200e+00 30    -60.099286  5.885211e-03 -60.093401 -60.169749   
1.141700e+01 31    -60.100140  6.662741e-03 -60.093478 -60.170689   
1.601470e+01 32    -60.101326  7.461691e-03 -60.093864 -60.171940   
2.246400e+01 33    -60.102922  8.294720e-03 -60.094627 -60.173578   
3.151050e+01 34    -60.105062  9.221779e-03 -60.095840 -60.175737   
4.420010e+01 35    -60.107967  1.033319e-02 -60.097633 -60.178642   
6.200000e+01 36    -60.111913  1.168611e-02 -60.100227 -60.182577   
8.577500e+01 37    -60.117034  1.327492e-02 -60.103759 -60.187692   
1.095500e+02 38    -60.122163  1.479036e-02 -60.107372 -60.192812   
1.333250e+02 39    -60.127427  1.633215e-02 -60.111095 -60.197996   
1.571000e+02 40    -60.132819  1.789733e-02 -60.114922 -60.203116   
1.808750e+02 41    -60.138445  1.955927e-02 -60.118886 -60.208372   
2.046500e+02 42    -60.144739  2.162874e-02 -60.123111 -60.214098   
2.284250e+02 43    -60.152245  2.459667e-02 -60.127648 -60.221039   
2.522000e+02 44    -60.161276  2.893444e-02 -60.132342 -60.229460   
2.759750e+02 45    -60.172229  3.532478e-02 -60.136904 -60.239673   
2.997500e+02 46    -60.185674  4.483962e-02 -60.140835 -60.252188   
3.235250e+02 47    -60.201506  5.803656e-02 -60.143470 -60.266964   
3.473000e+02 48    -60.219720  7.564197e-02 -60.144078 -60.283830   
3.710750e+02 49    -60.240526  9.878799e-02 -60.141738 -60.303079   
3.948500e+02 50    -60.264136  1.287916e-01 -60.135344 -60.325041   
4.186250e+02 51    -60.290746  1.671232e-01 -60.123623 -60.349865   
4.424000e+02 52    -60.320551  2.153858e-01 -60.105165 -60.377607   
4.661750e+02 53    -60.353353  2.744454e-01 -60.078908 -60.408213   
4.899500e+02 54    -60.389078  3.454133e-01 -60.043664 -60.441726   
5.137250e+02 55    -60.4

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')